In [111]:
import pandas as pd
import numpy as np


In [112]:
from datetime import datetime,timedelta
def round_timestamp_30interval(x):
    if type(x)==str:
        x = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    return x - timedelta(minutes=x.minute%30,seconds = x.second,microseconds=x.microsecond)

def time_features(data):
    data['mins'] = data.ts.dt.minute
    data['hour'] = data.ts.dt.hour
    data['day'] = data.ts.dt.day
    data['month'] = data.ts.dt.month
    data['dayofweek'] = data.ts.dt.dayofweek
    data['quarter'] = data.ts.dt.quarter
    return data

def prediction_with_lag(df):
    return predict_with_lag.predict(df[['pickup_cluster', 'mins', 'hour', 'month', 'quarter',
           'dayofweek', 'lag_1', 'lag_2', 'lag_3']])

def shift_with_lag_and_rollingmean(df):
    df = df.sort_values(by=['pickup_cluster', 'ts']).drop_duplicates(subset=['ts','pickup_cluster'])
    df = df.set_index(['ts', 'pickup_cluster', 'mins', 'hour', 'month', 'quarter', 'dayofweek'])
    df['lag_1'] = df.groupby(level=['pickup_cluster'])['request_count'].shift(1)
    df['lag_2'] = df.groupby(level=['pickup_cluster'])['request_count'].shift(2)
    df['lag_3'] = df.groupby(level=['pickup_cluster'])['request_count'].shift(3)
    #df['rolling_mean'] = df.groupby(level=['pickup_cluster'])['request_count'].apply(lambda x: x.rolling(window = 3).mean()).shift(1)

    df = df.reset_index(drop = False).dropna()
    df = df[['ts', 'pickup_cluster', 'mins', 'hour', 'month', 'quarter',
           'dayofweek', 'lag_1', 'lag_2', 'lag_3','request_count']]
    return df

In [113]:
##testing data
df = pd.read_csv('./cleaned_test_booking_data.csv',compression='gzip')

In [114]:
import pickle
with open('./cluster_model/cluster.pkl','rb') as f:
    cluster_model = pickle.load(f)

In [115]:
with open('model.pkl','rb') as f:
    model = pickle.load(f)

## Preprocessing the prediction data ,as done for training data

### K-Mean Cluster

In [116]:
df['pickup_cluster'] = cluster_model.predict(df[['pick_lat','pick_lng']])
df

C:\Users\himan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MiniBatchKMeans was fitted without feature names
  warnings.warn(


,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,pickup_cluster
0,2021-03-26 06:49:38,-1,12.903468,77.637080,12.916259,77.675476,6
1,2021-03-26 15:14:23,0,12.903838,77.591774,12.890039,77.593720,14
2,2021-03-26 15:57:32,6,12.963516,77.674740,12.912828,77.627310,10
3,2021-03-26 23:34:53,7,12.944017,77.564270,12.967625,77.608060,40
4,2021-03-26 23:45:56,9,12.983270,77.752070,12.963221,77.748400,44
...,...,...,...,...,...,...,...
15288,2021-03-26 10:31:37,99975,12.907254,77.605490,12.926352,77.609024,5
15289,2021-03-26 15:37:32,99985,12.974960,77.632454,12.956863,77.619446,13
15290,2021-03-26 05:27:27,99989,12.919729,77.643160,12.894464,77.616560,21
15291,2021-03-26 18:12:56,99998,12.968055,77.574684,12.956118,77.567505,20


In [117]:
df['ts'] = df['ts'].apply(round_timestamp_30interval)

In [118]:
(df.groupby(['ts','pickup_cluster']).count()).reset_index()

,ts,pickup_cluster,number,pick_lat,pick_lng,drop_lat,drop_lng
0,2021-03-26 00:00:00,0,2,2,2,2,2
1,2021-03-26 00:00:00,5,11,11,11,11,11
2,2021-03-26 00:00:00,6,4,4,4,4,4
3,2021-03-26 00:00:00,9,3,3,3,3,3
4,2021-03-26 00:00:00,10,4,4,4,4,4
...,...,...,...,...,...,...,...
1720,2021-03-26 23:30:00,43,13,13,13,13,13
1721,2021-03-26 23:30:00,44,9,9,9,9,9
1722,2021-03-26 23:30:00,45,7,7,7,7,7
1723,2021-03-26 23:30:00,46,3,3,3,3,3


In [119]:
df = (df.groupby(['ts','pickup_cluster']).count()).reset_index()

df = df[['ts','number','pickup_cluster']]
df=df.groupby(by = ['ts','pickup_cluster']).count().reset_index()
df.columns = ['ts','pickup_cluster','request_count']

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725 entries, 0 to 1724
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ts              1725 non-null   datetime64[ns]
 1   pickup_cluster  1725 non-null   int32         
 2   request_count   1725 non-null   int64         
dtypes: datetime64[ns](1), int32(1), int64(1)
memory usage: 33.8 KB


In [121]:
l = [datetime(2021,3,26,00,00,00) + timedelta(minutes = 30*i) for i in range(0,51)]
lt = []
for x in l:
    lt.append([x, -1, 0])
temp = pd.DataFrame(lt, columns = ['ts','pickup_cluster','request_count'])
df = pd.concat([df,temp],ignore_index=True)

data = df.set_index(['ts', 'pickup_cluster']).unstack().fillna(value=0).asfreq(freq='30Min').stack().sort_index(level=1).reset_index()

# Removing Dummy Cluster
data = data[data.pickup_cluster>=0]

df = time_features(data)

C:\Users\himan\AppData\Local\Temp\ipykernel_8128\2448264648.py:8: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = df.set_index(['ts', 'pickup_cluster']).unstack().fillna(value=0).asfreq(freq='30Min').stack().sort_index(level=1).reset_index()


In [123]:
df[df['ts'] >= datetime(2021,3,27,00,00,00)]

,ts,pickup_cluster,request_count,mins,hour,day,month,dayofweek,quarter
99,2021-03-27 00:00:00,0,0.0,0,0,27,3,5,1
100,2021-03-27 00:30:00,0,0.0,30,0,27,3,5,1
101,2021-03-27 01:00:00,0,0.0,0,1,27,3,5,1
150,2021-03-27 00:00:00,1,0.0,0,0,27,3,5,1
151,2021-03-27 00:30:00,1,0.0,30,0,27,3,5,1
...,...,...,...,...,...,...,...,...,...
2548,2021-03-27 00:30:00,48,0.0,30,0,27,3,5,1
2549,2021-03-27 01:00:00,48,0.0,0,1,27,3,5,1
2598,2021-03-27 00:00:00,49,0.0,0,0,27,3,5,1
2599,2021-03-27 00:30:00,49,0.0,30,0,27,3,5,1


In [ ]:
start_date = datetime(2021,3,27,00,00,00) 
from xgboost import XGBRegressor
for x in range(3):
    df = shift_with_lag_and_rollingmean(df)
    df.loc[df[df['ts']==start_date+timedelta(minutes=30*x)].index,'request_count'] = model.predict(df[df['ts']==start_date+timedelta(minutes=30*x)])

In [109]:
start_date = datetime(2021,3,27,00,00,00) 
from xgboost import XGBRegressor
for x in range(3):
    df = shift_with_lag_and_rollingmean(df)
    df2 = df[df['ts']==start_date+timedelta(minutes=30*x)]

In [110]:
df2

,ts,pickup_cluster,mins,hour,month,quarter,dayofweek,lag_1,lag_2,lag_3,request_count
44,2021-03-27 01:00:00,0,0,1,3,1,5,0.0,0.0,1.0,0.0
89,2021-03-27 01:00:00,1,0,1,3,1,5,0.0,0.0,0.0,0.0
134,2021-03-27 01:00:00,2,0,1,3,1,5,0.0,0.0,0.0,0.0
179,2021-03-27 01:00:00,3,0,1,3,1,5,0.0,0.0,0.0,0.0
224,2021-03-27 01:00:00,4,0,1,3,1,5,0.0,0.0,0.0,0.0
269,2021-03-27 01:00:00,5,0,1,3,1,5,0.0,0.0,1.0,0.0
314,2021-03-27 01:00:00,6,0,1,3,1,5,0.0,0.0,1.0,0.0
359,2021-03-27 01:00:00,7,0,1,3,1,5,0.0,0.0,0.0,0.0
404,2021-03-27 01:00:00,8,0,1,3,1,5,0.0,0.0,1.0,0.0
449,2021-03-27 01:00:00,9,0,1,3,1,5,0.0,0.0,1.0,0.0


In [83]:
X = df2.loc[:,['pickup_cluster', 'mins', 'hour', 'month', 'quarter', 'dayofweek', 'lag_1', 'lag_2', 'lag_3']]
y = df2['request_count']

In [84]:
X

,pickup_cluster,mins,hour,month,quarter,dayofweek,lag_1,lag_2,lag_3
39,0,0,0,3,1,5,1.0,1.0,1.0
81,1,0,0,3,1,5,0.0,0.0,0.0
123,2,0,0,3,1,5,0.0,0.0,0.0
165,3,0,0,3,1,5,0.0,1.0,1.0
207,4,0,0,3,1,5,0.0,0.0,0.0
249,5,0,0,3,1,5,1.0,1.0,1.0
291,6,0,0,3,1,5,1.0,1.0,1.0
333,7,0,0,3,1,5,0.0,0.0,1.0
375,8,0,0,3,1,5,1.0,1.0,1.0
417,9,0,0,3,1,5,1.0,1.0,1.0


In [85]:
from sklearn.metrics import root_mean_squared_error,r2_score
root_mean_squared_error(y_pred=model.predict(X),y_true=y)

0.4171641527445108

In [86]:
r2_score(y_pred=model.predict(X),y_true=y)

0.0

In [77]:
df

,ts,pickup_cluster,mins,hour,month,quarter,dayofweek,lag_1,lag_2,lag_3,request_count
3,2021-03-26 01:30:00,0,30,1,3,1,4,1.0,1.0,1.0,1.0
4,2021-03-26 02:00:00,0,0,2,3,1,4,1.0,1.0,1.0,0.0
5,2021-03-26 02:30:00,0,30,2,3,1,4,0.0,1.0,1.0,0.0
6,2021-03-26 03:00:00,0,0,3,3,1,4,0.0,0.0,1.0,1.0
7,2021-03-26 03:30:00,0,30,3,3,1,4,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2545,2021-03-26 23:00:00,49,0,23,3,1,4,1.0,1.0,1.0,1.0
2546,2021-03-26 23:30:00,49,30,23,3,1,4,1.0,1.0,1.0,1.0
2547,2021-03-27 00:00:00,49,0,0,3,1,5,1.0,1.0,1.0,0.0
2548,2021-03-27 00:30:00,49,30,0,3,1,5,0.0,1.0,1.0,0.0


In [89]:
df2

,ts,pickup_cluster,mins,hour,month,quarter,dayofweek,lag_1,lag_2,lag_3,request_count
39,2021-03-27,0,0,0,3,1,5,1.0,1.0,1.0,0.0
81,2021-03-27,1,0,0,3,1,5,0.0,0.0,0.0,0.0
123,2021-03-27,2,0,0,3,1,5,0.0,0.0,0.0,0.0
165,2021-03-27,3,0,0,3,1,5,0.0,1.0,1.0,0.0
207,2021-03-27,4,0,0,3,1,5,0.0,0.0,0.0,0.0
249,2021-03-27,5,0,0,3,1,5,1.0,1.0,1.0,0.0
291,2021-03-27,6,0,0,3,1,5,1.0,1.0,1.0,0.0
333,2021-03-27,7,0,0,3,1,5,0.0,0.0,1.0,0.0
375,2021-03-27,8,0,0,3,1,5,1.0,1.0,1.0,0.0
417,2021-03-27,9,0,0,3,1,5,1.0,1.0,1.0,0.0


In [93]:
pd.Series(y) 

39      0.0
81      0.0
123     0.0
165     0.0
207     0.0
249     0.0
291     0.0
333     0.0
375     0.0
417     0.0
459     0.0
501     0.0
543     0.0
585     0.0
627     0.0
669     0.0
711     0.0
753     0.0
795     0.0
837     0.0
879     0.0
921     0.0
963     0.0
1005    0.0
1047    0.0
1089    0.0
1131    0.0
1173    0.0
1215    0.0
1257    0.0
1299    0.0
1341    0.0
1383    0.0
1425    0.0
1467    0.0
1509    0.0
1551    0.0
1593    0.0
1635    0.0
1677    0.0
1719    0.0
1761    0.0
1803    0.0
1845    0.0
1887    0.0
1929    0.0
1971    0.0
2013    0.0
2055    0.0
2097    0.0
Name: request_count, dtype: float64

In [94]:
pd.Series(model.predict(X))

0     0.459225
1     0.049419
2     0.041066
3     0.273893
4     0.029849
5     0.594580
6     0.539427
7     0.139381
8     0.466004
9     0.469516
10    0.469516
11    0.143747
12    0.191507
13    0.475830
14    0.471058
15    0.469916
16    0.469916
17    0.469916
18    0.162401
19    0.447773
20    0.462860
21    0.469249
22    0.469249
23    0.467825
24    0.431532
25    0.431532
26    0.202197
27    0.497736
28    0.500868
29    0.462780
30    0.608797
31    0.459119
32    0.459119
33    0.292400
34    0.444401
35    0.444401
36    0.532123
37    0.295093
38    0.469541
39    0.469541
40    0.469541
41    0.230385
42    0.443678
43    0.490809
44    0.495586
45    0.497689
46    0.497689
47    0.205562
48    0.090770
49    0.327370
dtype: float32

In [95]:
X

,pickup_cluster,mins,hour,month,quarter,dayofweek,lag_1,lag_2,lag_3
39,0,0,0,3,1,5,1.0,1.0,1.0
81,1,0,0,3,1,5,0.0,0.0,0.0
123,2,0,0,3,1,5,0.0,0.0,0.0
165,3,0,0,3,1,5,0.0,1.0,1.0
207,4,0,0,3,1,5,0.0,0.0,0.0
249,5,0,0,3,1,5,1.0,1.0,1.0
291,6,0,0,3,1,5,1.0,1.0,1.0
333,7,0,0,3,1,5,0.0,0.0,1.0
375,8,0,0,3,1,5,1.0,1.0,1.0
417,9,0,0,3,1,5,1.0,1.0,1.0
